In [ ]:
%cd ..
%env TOKENIZERS_PARALLELISM=false

In [ ]:
import torch
from src import DataLoader, PredictionDatasetForSiamese, SiameseTokenClassificationModel
from src.scoring import compute_score

In [ ]:
device = torch.device('cuda')
model_path = 'weights/v1-e0+.pt'
dataset_path = 'data/splitted/val.csv'
batch_size = 1

In [ ]:
model = SiameseTokenClassificationModel()
model.load_state_dict(torch.load(model_path))
model = model.requires_grad_(False).to(device)

In [ ]:
from tqdm.auto import tqdm

dataset = PredictionDatasetForSiamese(dataset_path, model.tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

answers = []
for xs, q, r, s, q_state, r_state in tqdm(dataloader):       
    q.__setstate__(q_state)
    r.__setstate__(r_state)
    
    q = q.to(device)
    r = r.to(device)
    s = s.to(device)

    q_special_tokens_mask = q.pop('special_tokens_mask')
    r_special_tokens_mask = r.pop('special_tokens_mask')

    preds = model(q, r, s)
    ans = model.decode_answers(xs, q, r, q_special_tokens_mask, r_special_tokens_mask, preds)
    answers.extend(ans)

In [ ]:
compute_score(answers)